In [14]:
import json
import re
from glob import glob
from tqdm.notebook import tqdm
import pandas as pd

import nglview
from pymatgen.core import Structure
from pymatgen.io.jarvis import JarvisAtomsAdaptor

In [3]:
!pwd

/Users/yk4r2/Documents/idao_22/adhoc


In [4]:
structure_paths = glob('../data/dichalcogenides_public/structures/*.json')
structures = {
    list(filter(lambda x: x != '', re.split(r'\/(\w+)\.json', path)))[-1]:
    Structure.from_dict(json.load(open(path, 'r')))
    for path in tqdm(structure_paths)
}

  0%|          | 0/2966 [00:00<?, ?it/s]

In [10]:
jarvis = {
    key: JarvisAtomsAdaptor.get_atoms(value)
    for key, value in structures.items()
}

In [19]:
jarvis[list(jarvis.keys())[1]]

Mo63WSeS26
1.0
25.5225256 0.0 1.5628039641098191e-15
-12.761262799999994 22.10315553833868 1.5628039641098191e-15
0.0 0.0 14.879004
Mo W Se S
63 1 1 126
Cartesian
5.442793000966953e-07 1.8419299263228 3.71975
11.1661080442793 4.6048249263228 3.71975
9.570950501741766 7.367719999999999 3.71975
7.975793044279301 10.1306149263228 3.71975
6.380635544279301 12.8935099263228 3.71975
4.785478044279301 15.656404926322798 3.71975
3.1903205442793 18.4192999263228 3.71975
1.5951630442793003 21.182194926322797 3.71975
15.9515817092042 1.8419299263228 3.71975
14.3564242092042 4.6048249263228 3.71975
12.761266666666666 7.367719999999999 3.71975
11.1661092092042 10.1306149263228 3.71975
9.5709517092042 12.8935099263228 3.71975
7.9757942092042 15.656404926322798 3.71975
12.7612655442793 1.8419299263228 3.71975
6.3806367092042 18.4192999263228 3.71975
19.141898044279298 1.8419299263228 3.71975
17.5467405442793 4.6048249263228 3.71975
15.951583001741765 7.367719999999999 3.71975
14.356425544279299 10.13

In [23]:
targets = pd.read_csv('../data/dichalcogenides_public/targets.csv', index_col='_id')

In [24]:
targets

,band_gap
_id,
6141cf0f51c1cbd9654b8870,1.0843
6141cf1051c1cbd9654b8872,1.1102
6141cf11cc0e69a0cf28ab35,1.1484
6141cf11b842c2e72e2f2d48,1.8068
6141cf11ae4fb853db2e3f14,0.3600
...,...
6146d0b54e27a1844a5f0b02,1.1461
6146dd853ac25c70a5c6cdeb,0.3550
6146e9103ac25c70a5c6cded,0.3491


In [56]:
preds = []
for path in glob('../predictions/*.json'):
    pred_col = path.split('/')[-1].split('_pred.json')[0]
    prediction = pd.read_json(path)
    prediction = prediction.set_index('id').rename({'pred': pred_col}, axis=1)
    preds.append(prediction)

In [57]:
preds = pd.concat(preds, axis=1)
preds.head()

,jv_optb88vdw_bandgap_alignn,model_4,model_5,jv_mbj_bandgap_alignn
id,,,,
614205f93ac25c70a5c6ca6f,-0.031226,0.068248,0.224496,0.804508
6141f58c9cbada84a8676b95,-0.024849,1.427974,1.418460,0.819351
6141d3d2ee0a3fd43fb47b55,-0.031724,0.393946,0.491632,0.828986
6142298d9cbada84a8676cb5,-0.025576,1.425428,1.429869,0.817452
6141d43abaaf234b352902fc,-0.031713,1.102333,1.120952,0.807827


In [58]:
preds['predictions'] = preds[['model_4', 'model_5']].sum(axis=1) / 2

In [60]:
preds['predictions'] = preds[['jv_optb88vdw_bandgap_alignn', 'jv_mbj_bandgap_alignn']].sum(axis=1) / 2

In [64]:
preds['predictions'] = preds['jv_mbj_bandgap_alignn']

In [53]:
preds['predictions'] = preds.sum(axis=1) / 4

In [65]:
preds['predictions'].to_csv('../answer.csv')